In [ ]:
# ------------------------------------------------------------
# 05_bigquery_validation.ipynb
# Purpose: Validate uploaded BigQuery tables for consistency
# ------------------------------------------------------------

from google.cloud import bigquery
import os
import pandas as pd

# ✅ Set path to your downloaded service account key (JSON)
SERVICE_ACCOUNT_PATH = "../credentials/monzo-data-uploader-d16b82e5caaf.json"

# ✅ Set Google project and dataset IDs
PROJECT_ID = "monzo-data-uploader"
DATASET_ID = "monzo_reviews"

# Authenticate (locally)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SERVICE_ACCOUNT_PATH

# Create BigQuery client
client = bigquery.Client(project=PROJECT_ID)

print(f"✅ Connected to BigQuery project: {PROJECT_ID}")


In [ ]:
import sys
print(sys.executable)
!{sys.executable} -m pip install db-dtypes --upgrade --quiet


In [ ]:
# List tables inside the dataset
tables = list(client.list_tables(f"{PROJECT_ID}.{DATASET_ID}"))
print(f"📂 Tables found in dataset '{DATASET_ID}':")
for t in tables:
    print(f" - {t.table_id}")


In [ ]:
query = f"""
SELECT 'FactReviews' AS table_name, COUNT(*) AS row_count FROM `{PROJECT_ID}.{DATASET_ID}.FactReviews`
UNION ALL
SELECT 'DimPlatform', COUNT(*) AS row_count FROM `{PROJECT_ID}.{DATASET_ID}.DimPlatform`
UNION ALL
SELECT 'DimVersion', COUNT(*) AS row_count FROM `{PROJECT_ID}.{DATASET_ID}.DimVersion`
UNION ALL
SELECT 'DimDate', COUNT(*) AS row_count FROM `{PROJECT_ID}.{DATASET_ID}.DimDate`
UNION ALL
SELECT 'DimSentiment', COUNT(*) AS row_count FROM `{PROJECT_ID}.{DATASET_ID}.DimSentiment`
"""

In [ ]:
row_counts = client.query(query).to_dataframe()
display(row_counts)


In [ ]:
query = f"""
SELECT
  f.platform_id,
  p.platform,
  COUNT(*) AS review_count
FROM `{PROJECT_ID}.{DATASET_ID}.FactReviews` f
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DimPlatform` p
  ON f.platform_id = p.platform_id
GROUP BY 1, 2
ORDER BY review_count DESC
"""
platform_check = client.query(query).to_dataframe()
display(platform_check)


In [ ]:
validation_query = f"""
-- 1️⃣ Check foreign key consistency (Fact vs Dims)
SELECT
  'Platform Mismatch' AS issue,
  COUNT(*) AS problem_rows
FROM `{PROJECT_ID}.{DATASET_ID}.FactReviews` f
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DimPlatform` p
  ON f.platform_id = p.platform_id
WHERE p.platform_id IS NULL

UNION ALL
SELECT
  'Version Mismatch',
  COUNT(*)
FROM `{PROJECT_ID}.{DATASET_ID}.FactReviews` f
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DimVersion` v
  ON f.version_id = v.version_id
WHERE v.version_id IS NULL

UNION ALL
SELECT
  'Date Mismatch',
  COUNT(*)
FROM `{PROJECT_ID}.{DATASET_ID}.FactReviews` f
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DimDate` d
  ON f.date_id = d.date_id
WHERE d.date_id IS NULL

UNION ALL
SELECT
  'Sentiment Mismatch',
  COUNT(*)
FROM `{PROJECT_ID}.{DATASET_ID}.FactReviews` f
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.DimSentiment` s
  ON f.sentiment_id = s.sentiment_id
WHERE s.sentiment_id IS NULL;
"""

validation_results = client.query(validation_query).to_dataframe()
display(validation_results)
